In [ ]:
import urllib.request
import sqlite3
from html.parser import HTMLParser
from contextlib import contextmanager

In [ ]:
url = 'https://pokemondb.net/pokedex/all'
req = urllib.request.Request(url, headers={'user-Agent' : 'Mozilla'})

with urllib.request.urlopen(req) as html:
    page = html.read()
    page = str(page)

In [ ]:
# headers -> <div> -- class="sortwrap"

# Dex_Numbers -> <span> -- class="infocard-cell-data"

# Names -> <span> -- data-alt

# Base Stat Totals -> <td> -- class="cell-total"

# Other Stats -> <td> -- class="cell-num"

In [ ]:
class ParseStartTag(HTMLParser):
    def __init__(self, wanted_tag, wanted_attr_key):
        super().__init__()
        self.wanted_tag = wanted_tag
        self.wanted_attr_key = wanted_attr_key
        self.data = []
        
    def handle_starttag(self, tag, attrs):
        if tag == self.wanted_tag:
            for key, value in attrs:
                if key == self.wanted_attr_key:
                    self.data.append(value)

In [ ]:
class ParseText(HTMLParser):
    def __init__(self, wanted_tag, wanted_attrs):
        super().__init__()
        self.wanted_tag = wanted_tag
        self.wanted_attrs = wanted_attrs
        self.flag = False
        self.data = []
        
    def handle_starttag(self, tag, attrs):
        if tag == self.wanted_tag and all(attr in attrs for attr in self.wanted_attrs.items()):
            self.flag = True
            
    def handle_data(self, data):
        if self.flag == True:
            self.data.append(data)
            
    def handle_endtag(self, tag):
        if tag == self.wanted_tag:
            self.flag = False

### Headers

In [ ]:
header_obj = ParseText('div', {'class' : 'sortwrap'})
header_obj.feed(page)
headers = header_obj.data

In [ ]:
headers.remove('Type')
headers[0], headers[6], headers[7] = 'Dex_Num', 'Sp_Atk', 'Sp_Def'

### Names

In [ ]:
names_obj = ParseStartTag('span', 'data-alt')
names_obj.feed(page)
names = names_obj.data

In [ ]:
names = [name.removesuffix('icon').strip() for name in names]

### Dex_Nums

In [ ]:
dex_num = ParseText('span', {'class' : 'infocard-cell-data'})
dex_num.feed(page)
dex_nums = dex_num.data

### Base Totals

In [ ]:
totals = ParseText('td', {'class' : 'cell-total'})
totals.feed(page)
stat_totals = totals.data

### Other Stats

In [ ]:
stats_list = ParseText('td', {'class' : 'cell-num'})
stats_list.feed(page)
stats_lists_all = stats_list.data

In [ ]:
# Create sublist of lists
full = []
temp = []
for stat in stats_lists_all:
    temp.append(stat)
    if len(temp) == 6:
        full.append(list(temp))
        temp.clear()

In [ ]:
rows = list(map(list, zip(dex_nums, names, stat_totals)))

In [ ]:
iter_full = iter(full)
for pokemon in rows:
    pokemon.extend(next(iter_full))

### SQL TIME

In [ ]:
with sqlite3.connect('./PokemonSQL.sqlite') as con:
    cur = con.cursor()
    
    cur.execute(f'''CREATE TABLE pokemon(
                {headers[0]} TEXT,
                {headers[1]} TEXT,
                {headers[2]} INTEGER,
                {headers[3]} INTEGER,
                {headers[4]} INTEGER,
                {headers[5]} INTEGER,
                {headers[6]} INTEGER,
                {headers[7]} INTEGER,
                {headers[8]} INTEGER)
    ''')
    
    query = f'INSERT INTO pokemon ({",".join(headers)}) VALUES ({",".join("?" * len(headers))})'
    
    for row in rows:
        cur.execute(query, row)
        
    con.commit()
    cur.close()

### Queries

In [ ]:
@contextmanager
def Query(query, database='./PokemonSQL.sqlite'):
    try:
        con = sqlite3.connect(database)
        cur = con.cursor()
        yield cur.execute(query)
    finally:
        cur.close()
        con.close()

In [ ]:
# Get all pokemon with Mega in the name
query = '''SELECT * FROM pokemon WHERE Name LIKE "Mega %";'''
with Query(query) as finder:
    result = finder.fetchall()

In [ ]:
result

In [ ]:
# Get pokemon with the highest HP
query = '''SELECT Name, HP
            FROM pokemon 
            WHERE HP >= 200
            ORDER BY HP
            DESC;'''

In [ ]:
with Query(query) as finder:
    result = finder.fetchall()

In [ ]:
result